In [1]:
# dependencies and setup

import pandas as pd
import requests
import gmaps

# import API key

from config import g_key

# configure gmaps API key

gmaps.configure(api_key=g_key)

In [2]:
# 2 import the WeatherPy_database.csv file

city_data_df = pd.read_csv(r"..\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.15,52,82,1.63,broken clouds
1,1,Hermanus,ZA,-34.4187,19.2345,54.81,89,98,7.34,overcast clouds
2,2,New Norfolk,AU,-42.7826,147.0587,62.71,68,64,1.01,broken clouds
3,3,Chokurdakh,RU,70.6333,147.9167,32.95,68,91,13.49,overcast clouds
4,4,Porto Belo,BR,-27.1578,-48.5531,58.15,92,100,7.25,overcast clouds


In [3]:
# prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("Please enter minimum temperature you'd like for this trip"))
max_temp = float(input("Please enter maximum temperature you'd like for this trip"))

Please enter minimum temperature you'd like for this trip75
Please enter maximum temperature you'd like for this trip90


In [4]:
# filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Santa Helena De Goias,BR,-17.8136,-50.5969,79.12,57,92,4.07,overcast clouds
14,14,Ojinaga,MX,29.5667,-104.4167,83.35,42,40,12.66,scattered clouds
19,19,Katsuura,JP,35.1333,140.3000,77.58,65,4,7.40,clear sky
20,20,La Asuncion,VE,11.0333,-63.8628,82.13,89,74,12.66,broken clouds
23,23,Ciudad Bolivar,VE,8.1222,-63.5497,78.58,84,100,4.07,overcast clouds
26,26,Atuona,PF,-9.8000,-139.0333,77.40,76,27,20.33,scattered clouds
28,28,Ialibu,PG,-6.2821,143.9935,75.83,33,100,5.37,overcast clouds
29,29,Aden,YE,12.7794,45.0367,88.23,66,77,7.61,broken clouds
30,30,Marsh Harbour,BS,26.5412,-77.0636,84.02,70,15,9.19,few clouds
31,31,Saint George,US,37.1041,-113.5841,88.39,13,0,6.91,clear sky


In [5]:
# determine if there are any empty rows

preferred_cities_df.count()

City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [6]:
# drop any empty rows and create a new dataframe that doesn’t have empty rows

clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [7]:
# create dataframe called hotel_df to store hotel names along with city, country, max temp, and coordinates

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# create an empty column named Hotel Name

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Santa Helena De Goias,BR,79.12,overcast clouds,-17.8136,-50.5969,
14,Ojinaga,MX,83.35,scattered clouds,29.5667,-104.4167,
19,Katsuura,JP,77.58,clear sky,35.1333,140.3000,
20,La Asuncion,VE,82.13,broken clouds,11.0333,-63.8628,
23,Ciudad Bolivar,VE,78.58,overcast clouds,8.1222,-63.5497,
26,Atuona,PF,77.40,scattered clouds,-9.8000,-139.0333,
28,Ialibu,PG,75.83,overcast clouds,-6.2821,143.9935,
29,Aden,YE,88.23,broken clouds,12.7794,45.0367,
30,Marsh Harbour,BS,84.02,few clouds,26.5412,-77.0636,
31,Saint George,US,88.39,clear sky,37.1041,-113.5841,


In [8]:
# set parameters to search for hotels with 5000 meters

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# iterate through the hotel dataframe

for index, row in hotel_df.iterrows():

    # fetch latitude and longitude from dataframe
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # set up the base URL for the Google Directions API to get JSON data
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # make API request and retrieve the JSON data from the hotel search
    
    hotels = requests.get(base_url, params=params).json()   
    
    # get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# adding dependency
import numpy as np

In [11]:
# drop the rows where there is no Hotel Name

# replace empty string values with NaN panda objects in the Hotel Name column

hotel_df['Hotel Name'].replace('',np.nan,inplace=True)

# drop the rows where there are NaN values in the Hotel Name column

hotel_df.dropna(subset=['Hotel Name'], inplace=True)

# create new dataframe to store location, weather data and nearest hotel

clean_hotel_df = hotel_df

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Santa Helena De Goias,BR,79.12,overcast clouds,-17.8136,-50.5969,Castro Castle Hotel
14,Ojinaga,MX,83.35,scattered clouds,29.5667,-104.4167,Suites Las Palmas
19,Katsuura,JP,77.58,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
20,La Asuncion,VE,82.13,broken clouds,11.0333,-63.8628,Colinas de Matasiete
23,Ciudad Bolivar,VE,78.58,overcast clouds,8.1222,-63.5497,Hotel Edi
26,Atuona,PF,77.40,scattered clouds,-9.8000,-139.0333,Villa Enata
28,Ialibu,PG,75.83,overcast clouds,-6.2821,143.9935,PITPIT PASPAS GUESTHOUSE
29,Aden,YE,88.23,broken clouds,12.7794,45.0367,Sama Emirate Hotel
30,Marsh Harbour,BS,84.02,few clouds,26.5412,-77.0636,Abaco Beach Resort
31,Saint George,US,88.39,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


In [12]:
# create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# export the city_data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# formatting template provided

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# get the data from each row in the clean_hotel_df dataframe, add it to the formatting template, and store the data in a list

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# get the latitude and longitude from each row and store in a new dataframe

locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# add a marker layer for each city to the map

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))